In [1]:
# 下載程式碼
!git clone https://github.com/taipingeric/ControlNet.git
# 移動到程式碼資料夾
%cd ControlNet

[WinError 2] 系統找不到指定的檔案。: 'ControlNet'
D:\Controlnet\ControlNet


In [ ]:
# 下載官方預訓練模型 StableDiffusion 2.1
!wget https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt

In [2]:
# 安裝相依套件
!pip install -r requirements.txt
!pip install -U datasets

In [ ]:
# 轉換官方模型 -> ControlNet 模型
!python tool_add_control_sd21.py ./v2-1_512-ema-pruned.ckpt ./control_sd21_ini.ckpt

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import json
import numpy as np
from torch.utils.data import Dataset
from datasets import load_dataset

In [4]:
# 載入資料集
ds = load_dataset("Kelvin878/PCB_V2")

In [5]:
class MyDataset(Dataset):

    def __init__(self, hf_dataset):
        self.data = hf_dataset
        self.img_size = 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        source = item['guide'].resize(
            (self.img_size, self.img_size)).convert('RGB')
        target = item['image'].resize((self.img_size, self.img_size))

        source = np.array(source)
        target = np.array(target).astype(np.float32)
        prompt = item['text']

        # binarize source image
        # Normalize source images to [0, 1].
        source = np.where(source > 127, 1, 0).astype(np.float32)
        # Normalize target images to [-1, 1].
        target = (target / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [6]:
dataset = MyDataset(ds['train'])
print(len(dataset))

item = dataset[1234]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

1290
An image of Plated Circuit Board with 2 excess solder defects
(512, 512, 3)
(512, 512, 3)


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
# from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict

# Configs
resume_path = './control_sd21_ini.ckpt'
batch_size = 2  # GPU 24GB RAM
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False

# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v21.yaml').cpu()  # 模型設定檔
model.load_state_dict(load_state_dict(resume_path, location='cpu'))  # 讀取模型權重
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

# Misc
dataset = MyDataset(ds['train'])
dataloader = DataLoader(dataset,
                        num_workers=0,
                        batch_size=batch_size,
                        shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])

# Train!
trainer.fit(model, dataloader)

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v21.yaml]
Loaded state_dict from [./control_sd21_ini.ckpt]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\User\anaconda3\envs\controlnet\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
C:\Users\User\anaconda3\envs\controlnet\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:280: LightningDeprecationWarning: Base `LightningModule.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  rank_zero_deprecation(
C:\Users\User\anaconda3\envs\controlnet\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:287: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  ra

Training: 0it [00:00, ?it/s]

C:\Users\User\anaconda3\envs\controlnet\lib\site-packages\pytorch_lightning\utilities\data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Data shape for DDIM sampling is (2, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


...


DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:16<00:00,  3.10it/s]


Data shape for DDIM sampling is (2, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:16<00:00,  3.10it/s]


Data shape for DDIM sampling is (2, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:15<00:00,  3.14it/s]


訓練結果在 ./lightning_logs 資料夾中

訓練過程圖片在 ./image_log/train/ 資料夾中

#### Visulization

In [ ]:
from glob import glob
import matplotlib.pyplot as plt

paths = sorted(glob('./image_log/train/reconstruction*.png')) + sorted(
    glob('./image_log/train/control*.png'))

control_path = paths[-1]
reconstruction_path = control_path.replace('control', 'reconstruction')
text_path = control_path.replace('control', 'conditioning')
pred_path = control_path.replace('control', 'samples_cfg_scale_9.00')

plt.figure(figsize=(30, 30))
plt.subplot(3, 2, 1)
plt.imshow(plt.imread(reconstruction_path))

plt.axis('off')
plt.title('Control')
plt.subplot(3, 2, 2)
plt.imshow(plt.imread(reconstruction_path))
plt.imshow(plt.imread(control_path), alpha=0.5)
plt.axis('off')
plt.title('Reconstruction')

plt.subplot(3, 2, 3)
plt.imshow(plt.imread(pred_path))
plt.subplot(3, 2, 4)
plt.imshow(plt.imread(pred_path))
plt.imshow(plt.imread(control_path), alpha=0.5)
plt.subplot(3, 2, 5)
plt.imshow(plt.imread(text_path))
plt.subplot(3, 2, 6)
plt.imshow(plt.imread(control_path))
plt.tight_layout()
plt.show()